# Retraining DeepVariant v1.6.1 using Parabricks

#### What is Parabricks?

NVIDIA Parabricks® is the only GPU-accelerated computational genomics toolkit that delivers fast and accurate analysis for sequencing centers, clinical teams, genomics researchers, and next-generation sequencing instrument developers. Parabricks provides GPU-accelerated versions of tools used every day by computational biologists and bioinformaticians—enabling significantly faster runtimes, workflow scalability, and lower compute costs.

The toolkit includes full compatibility with workflow languages and managers (WDL, NextFlow, Cromwell) to easily intertwine GPU- and CPU-powered tasks, as well as support for easy cloud deployment (AWS, GCP, Terra, and DNAnexus).

#### What is DeepVariant?

[DeepVariant](https://www.nature.com/articles/nbt.4235.epdf?author_access_token=q4ZmzqvvcGBqTuKyKgYrQ9RgN0jAjWel9jnR3ZoTv0NuM3saQzpZk8yexjfPUhdFj4zyaA4Yvq0LWBoCYQ4B9vqPuv8e2HHy4vShDgEs8YxI_hLs9ov6Y1f_4fyS7kGZ), developed by Google, is a deep learning-based variant caller that takes aligned reads, produces pileup image tensors from them, classifies each tensor using a convolutional neural network, and then outputs the results in a VCF or gVCF file.

# Downloading the data

The example data for this notebook can be found on Google cloud and requires the [gsutil](https://cloud.google.com/storage/docs/gsutil) tool. We will keep it in a folder called `data`. In total it is ~14 GB and should take a few minutes to download. 

In [ ]:
%%sh 

DATA_BUCKET="gs://deepvariant/training-case-study/BGISEQ-HG001"
DATA_DIR="data"
OUTPUT_DIR="output"
LOG_DIR="logs"

mkdir -p ${DATA_DIR}
mkdir -p ${OUTPUT_DIR}
mkdir -p ${LOG_DIR}

gsutil -m cp "${DATA_BUCKET}/BGISEQ_PE100_NA12878.sorted.chr*.bam*" "${DATA_DIR}"
gsutil -m cp -r "${DATA_BUCKET}/ucsc_hg19.fa*" "${DATA_DIR}"
gsutil -m cp -r "${DATA_BUCKET}/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_*" "${DATA_DIR}"

# Building a dataset to retrain DeepVariant

To retrain the WGS baseline model, we need a dataset to train on. We will use Chromosome 1 from HG001 to generate a training dataset. 

Note: The filepaths for mounting data might have to change depending on where you cloned this repo. 

Note: This took ~7 minutes on two GPUs. 

In [2]:
%%sh

INPUT_DIR="/data"
OUTPUT_DIR="/output"
REF="${INPUT_DIR}/ucsc_hg19.fa"
BAM_CHR1="${INPUT_DIR}/BGISEQ_PE100_NA12878.sorted.chr1.bam"
TRUTH_VCF="${INPUT_DIR}/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer_chrs_FIXED.vcf.gz"
TRUTH_BED="${INPUT_DIR}/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_nosomaticdel_chr.bed"
TRAIN_EXAMPLES="${OUTPUT_DIR}/training_set_gpu.with_label.tfrecord.gz"
CONTAINER="nvcr.io/nvidia/clara/deepvariant_train:4.3.2-1"

docker run \
    --gpus all \
    --rm \
    -v ${PWD}/data:/data \
    -v ${PWD}/output:/output \
    ${CONTAINER} pbrun make_examples \
    --ref ${REF} \
    --reads ${BAM_CHR1} \
    --num-streams-per-gpu 4 \
    --num-gpus 2 \
    --num-cpu-threads-per-stream 8 \
    -L "chr1" \
    --disable-use-window-selector-model \
    --truth-variants ${TRUTH_VCF} \
    --confident-regions ${TRUTH_BED} \
    --examples ${TRAIN_EXAMPLES} \
    --channel-insert-size


/usr/local/parabricks/binaries//bin/make_examples /data/ucsc_hg19.fa /data/BGISEQ_PE100_NA12878.sorted.chr1.bam 2 4 -n 8 -z 4 --mode training --truth_variants /data/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer_chrs_FIXED.vcf.gz --confident_regions /data/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_nosomaticdel_chr.bed --examples /output/training_set_gpu.with_label.tfrecord.gz -disable-use-window-selector-model -L chr1 --channel_insert_size --pileup_image_width 221 --max_reads_per_partition 1500 --partition_size 1000 --vsc_min_count_snps 2 --vsc_min_count_indels 2 --vsc_min_fraction_snps 0.12 --min_mapping_quality 5 --min_base_quality 10 --alt_aligned_pileup none --variant_caller VERY_SENSITIVE_CALLER --dbg_min_base_quality 15 --ws_min_windows_distance 80 --aux_fields_to_keep HP --p_error 0.001 --max_ins_size 10
Please visit https://docs.nvidia.com/clara/#parabricks for detailed docum

[PB Info 2024-Sep-03 17:58:04] ------------------------------------------------------------------------------
[PB Info 2024-Sep-03 17:58:04] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2024-Sep-03 17:58:04] ||                      Version dv_training_1.6.1.test                      ||
[PB Info 2024-Sep-03 17:58:04] ||                               make_examples                              ||
[PB Info 2024-Sep-03 17:58:04] ------------------------------------------------------------------------------
[PB Info 2024-Sep-03 17:58:04] Starting DeepVariant
[PB Info 2024-Sep-03 17:58:04] Running with 2 GPU devices, each with 4 group instances and 8 workers
[W::hts_idx_load2] The index file is older than the data file: /data/BGISEQ_PE100_NA12878.sorted.chr1.bam.bai
[W::hts_idx_load2] The index file is older than the data file: /data/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer_chrs_FIXED

We will use Chromosome 21 from HG001 to generate a validation dataset. 

Note: This took ~3 minutes on 2 GPUs. 

In [4]:
%%sh 

INPUT_DIR="/data"
OUTPUT_DIR="/output"
REF="${INPUT_DIR}/ucsc_hg19.fa"
BAM_CHR21="${INPUT_DIR}/BGISEQ_PE100_NA12878.sorted.chr21.bam"
TRUTH_VCF="${INPUT_DIR}/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer_chrs_FIXED.vcf.gz"
TRUTH_BED="${INPUT_DIR}/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_nosomaticdel_chr.bed"
VAL_EXAMPLES="${OUTPUT_DIR}/validation_set_gpu.with_label.tfrecord.gz"
CONTAINER="nvcr.io/nvidia/clara/deepvariant_train:4.3.2-1"

docker run \
    --gpus all \
    --rm \
    -v ${PWD}/data:/data \
    -v ${PWD}/output:/output \
    ${CONTAINER} pbrun make_examples \
    --ref ${REF} \
    --reads ${BAM_CHR21} \
    --num-streams-per-gpu 4 \
    --num-gpus 2 \
    --num-cpu-threads-per-stream 8 \
    -L "chr21" \
    --disable-use-window-selector-model \
    --truth-variants ${TRUTH_VCF} \
    --confident-regions ${TRUTH_BED} \
    --examples ${VAL_EXAMPLES} \
    --channel-insert-size

/usr/local/parabricks/binaries//bin/make_examples /data/ucsc_hg19.fa /data/BGISEQ_PE100_NA12878.sorted.chr21.bam 2 4 -n 8 -z 4 --mode training --truth_variants /data/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer_chrs_FIXED.vcf.gz --confident_regions /data/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_nosomaticdel_chr.bed --examples /output/validation_set_gpu.with_label.tfrecord.gz -disable-use-window-selector-model -L chr21 --channel_insert_size --pileup_image_width 221 --max_reads_per_partition 1500 --partition_size 1000 --vsc_min_count_snps 2 --vsc_min_count_indels 2 --vsc_min_fraction_snps 0.12 --min_mapping_quality 5 --min_base_quality 10 --alt_aligned_pileup none --variant_caller VERY_SENSITIVE_CALLER --dbg_min_base_quality 15 --ws_min_windows_distance 80 --aux_fields_to_keep HP --p_error 0.001 --max_ins_size 10
Please visit https://docs.nvidia.com/clara/#parabricks for detailed d

[PB Info 2024-Sep-03 18:08:41] ------------------------------------------------------------------------------
[PB Info 2024-Sep-03 18:08:41] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2024-Sep-03 18:08:41] ||                      Version dv_training_1.6.1.test                      ||
[PB Info 2024-Sep-03 18:08:41] ||                               make_examples                              ||
[PB Info 2024-Sep-03 18:08:41] ------------------------------------------------------------------------------
[PB Info 2024-Sep-03 18:08:41] Starting DeepVariant
[PB Info 2024-Sep-03 18:08:41] Running with 2 GPU devices, each with 4 group instances and 8 workers
[W::hts_idx_load2] The index file is older than the data file: /data/BGISEQ_PE100_NA12878.sorted.chr21.bam.bai
[W::hts_idx_load2] The index file is older than the data file: /data/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer_chrs_FIXE

### Shuffling the training data

Before we can train the model we will need to shuffle each set of examples and generate a data config file. This has to be done for both the training and validation dataset. 

In [5]:
%%sh

OUTPUT_DIR="/output"
CONTAINER="nvcr.io/nvidia/clara/deepvariant_train:4.3.2-1"
INPUT_FILES=$(ls ${PWD}/output/training_set_gpu.with_label.tfrecord-?????-of-00004.gz | sed 's|'${PWD}/output'|'${OUTPUT_DIR}'|')

# shuffle training set 
docker run \
    --gpus all \
    --rm \
    -v ${PWD}/output:${OUTPUT_DIR} \
    ${CONTAINER} pbrun shuffle \
    --input-pattern-list ${INPUT_FILES} \
    --output-pattern-prefix=${OUTPUT_DIR}/training_set_gpu.with_label.shuffled \
    --output-dataset-config=${OUTPUT_DIR}/training_set_gpu.pbtxt \
    --output-dataset-name="HG001" \
    --direct-num-workers=4

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation



[PB Info 2024-Sep-03 18:11:23] ------------------------------------------------------------------------------
[PB Info 2024-Sep-03 18:11:23] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2024-Sep-03 18:11:23] ||                      Version dv_training_1.6.1.test                      ||
[PB Info 2024-Sep-03 18:11:23] ||                                  shuffle                                 ||
[PB Info 2024-Sep-03 18:11:23] ------------------------------------------------------------------------------
[PB Info 2024-Sep-03 18:11:23] -> Reading input
[PB Info 2024-Sep-03 18:11:23] --> Processing /output/training_set_gpu.with_label.tfrecord-00000-of-00004.gz
[PB Info 2024-Sep-03 18:11:23] --> Processing /output/training_set_gpu.with_label.tfrecord-00002-of-00004.gz
[PB Info 2024-Sep-03 18:11:23] --> Processing /output/training_set_gpu.with_label.tfrecord-00001-of-00004.gz
[PB Info 2024-Sep-03 18:11:23] --> Processing /output/training_set_gpu.with

In [6]:
%%sh

OUTPUT_DIR="/output"
CONTAINER="nvcr.io/nvidia/clara/deepvariant_train:4.3.2-1"
INPUT_FILES=$(ls ${PWD}/output/validation_set_gpu.with_label.tfrecord-?????-of-00004.gz | sed 's|'${PWD}/output'|'${OUTPUT_DIR}'|')

# shuffle validation set
docker run \
    --gpus all \
    --rm \
    -v ${PWD}/output:/output \
    ${CONTAINER} pbrun shuffle \
    --input-pattern-list ${INPUT_FILES} \
    --output-pattern-prefix=${OUTPUT_DIR}/validation_set_gpu.with_label.shuffled \
    --output-dataset-config=${OUTPUT_DIR}/validation_set_gpu.pbtxt \
    --output-dataset-name="HG001" \
    --direct-num-workers=4

Please visit https://docs.nvidia.com/clara/#parabricks for detailed documentation



[PB Info 2024-Sep-03 18:22:32] ------------------------------------------------------------------------------
[PB Info 2024-Sep-03 18:22:32] ||                 Parabricks accelerated Genomics Pipeline                 ||
[PB Info 2024-Sep-03 18:22:32] ||                      Version dv_training_1.6.1.test                      ||
[PB Info 2024-Sep-03 18:22:32] ||                                  shuffle                                 ||
[PB Info 2024-Sep-03 18:22:32] ------------------------------------------------------------------------------
[PB Info 2024-Sep-03 18:22:32] -> Reading input
[PB Info 2024-Sep-03 18:22:32] --> Processing /output/validation_set_gpu.with_label.tfrecord-00000-of-00004.gz
[PB Info 2024-Sep-03 18:22:32] --> Processing /output/validation_set_gpu.with_label.tfrecord-00001-of-00004.gz
[PB Info 2024-Sep-03 18:22:32] --> Processing /output/validation_set_gpu.with_label.tfrecord-00002-of-00004.gz
[PB Info 2024-Sep-03 18:22:32] --> Processing /output/validation_set_

# Training the DeepVariant model

Next we want to run the following code block to train and evaluate the model and generate checkpoints. 

It will use the validation/tune dataset to evaluate the model at the end of each epoch and keep track of which checkpoint performs the best. 

In [7]:
%%sh

# all parameters below are used as an example. They are not optimized for this dataset, and are not recommended as the best default

BIN_VERSION="1.6.1"
OUTPUT_DIR="/output"
TRAINING_DIR="/training_dir"
LOG_DIR="logs"

( docker run --gpus 2 \
    --gpus all \
    --rm \
    -v ${PWD}/output:/output \
    -v ${PWD}/training_dir:/training_dir \
    -v ${PWD}/dv_config.py:/dv_config.py \
    google/deepvariant:"${BIN_VERSION}-gpu" \
    train \
    --config=/dv_config.py:base \
    --config.train_dataset_pbtxt="${OUTPUT_DIR}/training_set_gpu.pbtxt" \
    --config.tune_dataset_pbtxt="${OUTPUT_DIR}/validation_set_gpu.pbtxt" \
    --config.init_checkpoint=gs://deepvariant/models/DeepVariant/1.6.1/checkpoints/wgs/deepvariant.wgs.ckpt \
    --config.num_epochs=10 \
    --config.learning_rate=0.0001 \
    --config.num_validation_examples=0 \
    --experiment_dir=${TRAINING_DIR} \
    --strategy=mirrored \
    --config.batch_size=512 \
) > "${LOG_DIR}/train.log" 2>&1

# Choose the best model

We then want to pick the best mdoel. We can determine which model to use by running the line of code below. 

In [8]:
! cat training_dir/checkpoints/checkpoint | sed -n 's/model_checkpoint_path: "\(.*\)"/\1/p'

ckpt-6689
